# Deloitte Quantum Challenge 2023

# 0. References and credits

In [ ]:
'''
Team: Quantux
Members: ...

V00 20230317 - Establishment
V01 20230226 - Add various local & realtime VQE client
V02 20230301 - Add decorators and modularise pipeline
'''

In [ ]:
'''
References:
https://github.com/PacktPublishing/Quantum-Chemistry-and-Computing-for-the-Curious/blob/main/Chapter_05_Variational_Quantum_Eigensolver_.VQE._algorithm_V2.ipynb
https://qiskit.org/documentation/nature/tutorials/03_ground_state_solvers.html
https://dev.to/kcdchennai/python-decorator-to-measure-execution-time-54hk
https://qiskit.org/documentation/nature/_modules/qiskit_nature/second_q/
https://qiskit.org/documentation/nature/migration/00b_Electronic_structure_with_v0.5.html
https://qiskit.org/documentation/nature/tutorials/05_problem_transformers.html
https://qiskit.org/documentation/stubs/qiskit.providers.ibmq.AccountProvider.html
https://qiskit.org/documentation/partners/qiskit_ibm_provider/tutorials/Migration_Guide_from_qiskit-ibmq-provider.html

https://towardsdatascience.com/state-of-the-art-machine-learning-hyperparameter-optimization-with-optuna-a315d8564de1

'''

# 1. Import libraries and qiskit functions

In [1]:
import pandas as pd
import numpy as np
import warnings, os, time
import matplotlib.pyplot as plt
from tqdm import tqdm

warnings.filterwarnings('ignore')

from functools import wraps
from qiskit.utils import algorithm_globals

from functools import partial
from scipy.optimize import minimize

_SEED = 5
np.random.seed(_SEED)
algorithm_globals.random_seed = _SEED

In [2]:
# All are post-migration imports, ie. qiskit_nature == 0.5.0

# Molecule definition
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance

# Load for runtime qiskit
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider(instance="deloitte-event23/level-1-access/quantux")
# provider.backends()
qasm_simulator = provider.get_backend('ibmq_qasm_simulator')
statevector_simulator = provider.get_backend('simulator_statevector')


from qiskit_nature.runtime import VQEClient

# Driver & ansatz libraries
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import UCC, UCCSD, PUCCD, SUCCD, UVCCSD
from qiskit.quantum_info import Pauli

# Algorithms, mappers & optimizers
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter, BravyiKitaevMapper, ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer

from qiskit_nature.second_q.algorithms import NumPyMinimumEigensolverFactory, VQEUCCFactory, VQEUVCCFactory
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import TwoLocal, PauliTwoDesign, EfficientSU2, RealAmplitudes
#from qiskit.circuit.library.initial_states import hartree_fock, fermionic_gaussian_state, vscf
#from qiskit.circuit.library.initial_points import mp2_initial_point, hf_initial_point, mp2_initial_point, vscf_initial_point
from qiskit.algorithms.optimizers import SLSQP, SPSA, QNSPSA

from qiskit.algorithms import HamiltonianPhaseEstimation, PhaseEstimation
from qiskit.opflow import StateFn, PauliExpectation, CircuitSampler, PauliTrotterEvolution

# Estimators & Samplers
from qiskit.primitives import Estimator, Sampler


# 2. Define construct problem modules

# 2-0. Decorators

In [3]:
# Timeit decorator
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__} took {total_time:.4f} seconds to run\n')
        return result
    return timeit_wrapper


In [4]:
# Try except decorator
def get_err_msg(value):
    def decorate(f):
        def applicator(*args, **kwargs):
            try:
                print('{}: Loading...'.format(f.__name__))
                return f(*args,**kwargs)
                print('Success in loading {}'.format(f.__name__))
                
            except:
                print('Fail in loading {}'.format(f.__name__))
                return value
        return applicator
    return decorate

# 2-1. Define modular functions

In [5]:
# 1. Get problem driver
@get_err_msg('')
@timeit
def _1_get_problem_driver(display_dict, basis, molecule = None, moleinfo = None):
    
    # =============================
    # 1. Define Molecule dictionary and return problem from PySCFDriver
    # =============================
    
    # =============================
    if molecule:
        moleculeinfo = MoleculeInfo(symbols = molecule['symbols'], coords = molecule['coords'], 
                                    charge = molecule['charge'], multiplicity = molecule['multiplicity'])
        print(moleculeinfo)
    elif moleinfo:
        moleculeinfo = moleinfo
        print(moleculeinfo)
    
    driver = PySCFDriver.from_molecule(moleculeinfo, basis=basis)
    problem = driver.run()
    
    
    if display_dict is not None:
        display_dict['molecule'] = '{}'.format(molecule['symbols'])
        display_dict['charge'] = '{}'.format(molecule['charge'])
        display_dict['multiplicity'] = '{}'.format(molecule['multiplicity'])
        
        display_dict['reference_energy'] = '{}'.format(problem.reference_energy)
        display_dict['num_spin_orbitals'] = '{}'.format(problem.num_spin_orbitals)
        display_dict['num_spatial_orbitals'] = '{}'.format(problem.num_spatial_orbitals)
        display_dict['num_particles'] = '{}'.format(problem.num_particles)
        display_dict['nuclear_repulsion_energy'] = '{}'.format(problem.nuclear_repulsion_energy)
        display_dict['num_alpha'] = '{}'.format(problem.num_alpha)
        display_dict['num_beta'] = '{}'.format(problem.num_beta)
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
    
    return problem, display_dict_new

In [6]:
# 1. Get transform problem
@get_err_msg('')
@timeit
def _2_get_problem_transform(display_dict, problem, molecule, reduced):
    
    # =============================
    # 2. Transform the problem to reduce simulation space
    # =============================
    
    # =============================
    # Problem reduction
    if reduced == 'FreezeCore':
        try:
            fc_transformer = FreezeCoreTransformer(freeze_core = molecule['fc_transformer']['fc_freeze_core'], 
                                                   remove_orbitals = molecule['fc_transformer']['fc_remove_orbitals'])
            problem = fc_transformer.transform(problem)
            display_chosen = molecule['fc_transformer']['fc_remove_orbitals']
        except:
            print('FreezeCore Transformer did not succeed.')
            pass
        
        # How to determine which orbitals to be removed
        # https://quantumcomputing.stackexchange.com/questions/17852/use-one-body-integrals-to-know-which-orbitals-to-freeze-in-electronicstructurepr
        # https://www.youtube.com/watch?v=3B04KB0pDwE&t=667s
        
    elif reduced == 'ActiveSpace':
        #max_num_spatial_orbitals = problem.num_spatial_orbitals
        #max_num_electrons = problem.num_electrons
        #max_active_orbitals = itertools.combinations(max_num_spatial_orbitals, 2)
        # Use optuna to setup objective
        
        try:
            as_transformer = ActiveSpaceTransformer(num_electrons = molecule['as_transformer']['as_num_electrons'], 
                                                    num_spatial_orbitals = molecule['as_transformer']['as_num_spatial_orbitals'], 
                                                    active_orbitals = molecule['as_transformer']['as_active_orbitals'])
            problem = as_transformer.transform(problem)
            display_chosen = molecule['as_transformer']['as_active_orbitals']
        except:
            print('ActiveSpace Transformer did not succeed.')
            pass
    else:
        # print('Expect lengthy simulation if can not succeed in reducing orbitals using FreezeCoreTransformer or ActiveSpaceTransformer.')
        display_chosen = ''
        pass
    
    
    if display_dict is not None:
        display_dict['reduction_method'] = '{}'.format(reduced)
        display_dict['orbitals_removed'] = '{}'.format(display_chosen)
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
    
    return problem, display_dict_new

In [7]:
# 3. Get qubit operator
@get_err_msg('')
@timeit
def _3_get_qubit_operator(display_dict, problem, hyperparam, mapper_type):
    
    # =============================
    # 3. Define qubit mapping and convert to qubit operator
    # =============================
    
    # =============================
    # Qubit mapping
    if mapper_type == 'ParityMapper':
        mapper = ParityMapper()
    elif mapper_type == 'JordanWignerMapper':
        mapper = JordanWignerMapper()
    elif mapper_type == 'BravyiKitaevMapper':
        mapper = BravyiKitaevMapper()
    
    fermionic_hamiltonian = problem.hamiltonian
    second_q_op = fermionic_hamiltonian.second_q_op()
    
    num_particles = problem.num_particles
        
    qubit_converter = QubitConverter(mapper, 
                                     two_qubit_reduction = hyperparam['two_qubit_reduction'], 
                                     z2symmetry_reduction = hyperparam['z2symmetry_reduction'])
    qubit_op = qubit_converter.convert(second_q_op, num_particles = num_particles, sector_locator = problem.symmetry_sector_locator)
    
    
    if display_dict is not None:
        display_dict['second_q_op'] = '{}'.format("\n".join(str(second_q_op).splitlines()[:10] + ["..."]))
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
        
    return qubit_op, qubit_converter, display_dict_new

In [8]:
# 4. Get quantum problem solver
@get_err_msg('')
@timeit
def _4_get_ansatz(display_dict, problem, hyperparam, seed):
    
    # =============================
    # 4. Define various solver types with initalizing the ansatz or initial circuit
    # =============================
    
    # =============================
    # i. List of ansatz
    #ansatz_lst = [UCCSD(), UCC(), PUCCD(), SUCCD(), UVCCSD()] # Others like PUCCD needs alpha == beta, ie. lots of restrictions thus all except UCCSD are left unused
    idx_ansatz = 0
    #ansatz_chosen = ansatz_lst[rand_ansatz]
    ansatz_name = ['UCCSD']
    ansatz_chosen = UCCSD()
    
    # =============================
    # ii. For QNSPSA, must match num_qubits to the circuit observable, currently H2 is 4, use this constraint to subordinate for num_qubits' design from initial_circuits
    num_spin_orbitals = problem.num_spin_orbitals
    
    circuit_tl = TwoLocal(num_spin_orbitals, rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz', entanglement='full', reps=hyperparam['reps'], parameter_prefix = 'y')
    circuit_su2 = EfficientSU2(num_spin_orbitals, reps=hyperparam['reps'], entanglement="full")
    circuit_p2d = PauliTwoDesign(num_qubits=num_spin_orbitals, reps=hyperparam['reps'], seed=seed)
    circuit_ra = RealAmplitudes(num_qubits=num_spin_orbitals, reps=hyperparam['reps'])
    
    initial_circuit_lst = [circuit_tl, circuit_su2, circuit_p2d, circuit_ra]
    initial_circuit_name = ['TwoLocal', 'EfficientSU2', 'PauliTwoDesign', 'RealAmplitudes']
    idx_initial_circuit = np.random.randint(len(initial_circuit_lst))
    initial_circuit_chosen = initial_circuit_lst[idx_initial_circuit]
    
    # =============================
    # iii. Custom optimizers - A partial callable to scipy optimizer
    method_lst = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'trust-constr']
    opt = [partial(minimize, method=i) for i in method_lst]

    optimizer_lst = [SPSA(maxiter=100), SLSQP(maxiter=100)] + opt
    optimizer_name = ['SPSA', 'SLSQP'] + ['minimize_{}'.format(i) for i in method_lst]
    idx_optimizer = np.random.randint(len(optimizer_lst))
    optimizer_chosen = optimizer_lst[idx_optimizer]
    
    if display_dict is not None:
        display_dict['ansatz_chosen'] = ansatz_name[idx_ansatz]
        display_dict['initial_circuit_chosen'] = initial_circuit_name[idx_initial_circuit]
        display_dict['optimizer_chosen'] = optimizer_name[idx_optimizer]
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
    
    return ansatz_chosen, initial_circuit_chosen, optimizer_chosen, display_dict_new

In [9]:
# 5. Get optimizer
@get_err_msg('')
@timeit
def _5_get_solver_optimizer(display_dict, problem, hyperparam, qubit_op, qubit_converter, solver_type, ansatz_chosen, initial_circuit_chosen, optimizer_chosen, method):
    
    # =============================
    # 5. Define VQE solvers to solve for PES in quantum chemistry
    # =============================
    
    # a. Solver and optimizer fixed set
    # =============================
    if solver_type == 'vqe_qnspsa':
        sampler = Sampler()
        ansatz = initial_circuit_chosen
        initial_point = np.random.random(ansatz.num_parameters)
        
        fidelity = QNSPSA.get_fidelity(ansatz, sampler)
        
        estimator = Estimator()
        # loss function
        def qnspsa_loss(x):
            result = estimator.run([ansatz], [qubit_op], [x]).result()
            return np.real(result.values[0])

        # QNSPSA solver directly tie to optimize function to directly call VPE, not using GSES or QPE
        solver = QNSPSA(fidelity, maxiter=300, learning_rate=1e-2, perturbation=0.7)
        ground_state = solver.optimize(ansatz.num_parameters, qnspsa_loss, initial_point=initial_point)
        energy = ground_state[1]
        
    # =============================
    elif solver_type == 'vqe_runtime':
        ansatz = initial_circuit_chosen
        optimizer = optimizer_chosen
        
        solver = VQE(ansatz, optimizer, provider=provider, quantum_instance=qasm_simulator, shots=1024, measurement_error_mitigation=True)
        ground_state = solver.compute_minimum_eigenvalue(qubit_op)
        energy = vqe_calc.eigenvalue
        
    # =============================
    elif solver_type == 'vqe_runtime_with_noise':
        pass
    
    # b. Customizable Solvers and optimizers
    # =============================
    # Solvers
    elif solver_type == 'numpy_solver_with_filter':
        solver = NumPyMinimumEigensolverFactory(use_default_filter_criterion=True)
    # =============================
    elif solver_type == 'numpy_solver':
        solver = NumPyMinimumEigensolverFactory()
    # =============================
    elif solver_type == 'vqe_ansatz':
        estimator = Estimator()
        ansatz = ansatz_chosen
        optimizer = optimizer_chosen
        
        solver = VQEUCCFactory(estimator, ansatz, optimizer)
    # =============================
    elif solver_type == 'vqe_initial_circuit':
        estimator = Estimator()
        initial_circuit = initial_circuit_chosen
        optimizer = optimizer_chosen
        
        solver = VQE(estimator, initial_circuit, optimizer)
    
    # =============================
    # Optimizers
    if method == 'gses':
        calc = GroundStateEigensolver(qubit_converter, solver)
        ground_state = calc.solve(problem)
            
        energy = ground_state.total_energies[0]

    elif method == 'qpe':
        # Quantum Phase Estimation
        quantum_instance = QuantumInstance(backend = Aer.get_backend('aer_simulator_statevector'))
        evolution = PauliTrotterEvolution('trotter', reps = hyperparam['qpe_num_time_slices'])

        qpe = HamiltonianPhaseEstimation(hyperparam['qpe_n_ancilliae'], quantum_instance=quantum_instance)

        state_preparation = None
        # state_preparation = 
        ground_state = qpe.estimate(qubit_op, state_preparation, evolution=evolution)

        energy = ground_state.most_likely_eigenvalue

    
    if display_dict is not None:
        display_dict['solver'] = '{}'.format(solver_type)
        display_dict['solution_method'] = '{}'.format(method)
        
        display_dict_new = display_dict
    else:
        display_dict_new = display_dict
        
    return ground_state, energy, display_dict_new


# 2-2. Build construct pipeline

In [10]:
# Create construct problem
def get_construct_problem(molecule, hyperparam, moleinfo = None, display_report = False, reduced = 'ActiveSpace', basis = 'sto3g', mapper_type = 'JordanWignerMapper', 
                          solver_type = 'numpy_solver', method = 'gses', seed = _SEED):
    
    # =============================
    # 0. Display Report Dict
    # =============================
    if display_report == True:
        display_dict = {}
    else:
        display_dict = None
    
    # =============================
    # A. The quantum solver pipeline
    # =============================
    
    problem, display_dict_1 = _1_get_problem_driver(display_dict, basis, molecule, moleinfo) ## Check tonight
    
    problem, display_dict_2 = _2_get_problem_transform(display_dict_1, problem, molecule, reduced)
    
    qubit_op, qubit_converter, display_dict_3 = _3_get_qubit_operator(display_dict_2, problem, hyperparam, mapper_type)
    
    ansatz_chosen, initial_circuit_chosen, optimizer_chosen, display_dict_4 = _4_get_ansatz(display_dict_3, problem, hyperparam, seed)
    
    ground_state, energy, display_dict_5 = _5_get_solver_optimizer(display_dict_4, problem, hyperparam, 
                                                                   qubit_op, qubit_converter, solver_type, ansatz_chosen, initial_circuit_chosen, optimizer_chosen, method)
    
    if display_report == True:
        display_df = pd.DataFrame.from_dict([display_dict_5])
    else:
        display_df = None
    
    return ground_state, energy, display_df


In [11]:
# ===
gas_molecules = {
    'h2': {'symbols': ["H", "H"],
           'coords': [(0.0000, 0.0000, 0.0000),
                      (0.0000, 0.0000, 0.7414)],
           'multiplicity': 1,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [1, 1],
           #'atom_pair': (1, 2),
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           }
}

hyperparameters = {
    'h2': {'reps': 2,
           'two_qubit_reduction': True,
           'z2symmetry_reduction': 'auto',
           'perturbation_steps': np.linspace(-0.5, 0.5, 250),
           'qpe_num_time_slices': 1,
           'qpe_n_ancilliae': 3,
          },

}

# ===
molecule = gas_molecules['h2']
hyperparam = hyperparameters['h2']
# display_report = False
display_dict = {}
reduced = 'ActiveSpace'
basis = 'sto3g'
mapper_type = 'JordanWignerMapper'
solver_type = 'numpy_solver'
method = 'gses'


In [14]:
_, energy, display_df = get_construct_problem(molecule, hyperparam, moleinfo = None, display_report = True, reduced = 'ActiveSpace', basis = 'sto3g', 
                                              mapper_type = 'JordanWignerMapper', solver_type = 'numpy_solver', method = 'gses', seed = _SEED)
energy

_1_get_problem_driver: Loading...
Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Angstrom
	Geometry:
		H	(0.0, 0.0, 0.0)
		H	(0.0, 0.0, 0.7414)
Function _1_get_problem_driver took 0.9765 seconds to run

_2_get_problem_transform: Loading...
Function _2_get_problem_transform took 0.0121 seconds to run

_3_get_qubit_operator: Loading...
Function _3_get_qubit_operator took 0.0697 seconds to run

_4_get_ansatz: Loading...
Function _4_get_ansatz took 0.0066 seconds to run

_5_get_solver_optimizer: Loading...
Function _5_get_solver_optimizer took 0.1127 seconds to run



-1.1166843870853418

In [15]:
display_df

,molecule,charge,multiplicity,reference_energy,num_spin_orbitals,num_spatial_orbitals,num_particles,nuclear_repulsion_energy,num_alpha,num_beta,reduction_method,orbitals_removed,second_q_op,ansatz_chosen,initial_circuit_chosen,optimizer_chosen,solver,solution_method
0,"['H', 'H']",0,1,-1.1166843870853405,4,2,"(1, 1)",0.7137539936876182,1,1,ActiveSpace,"[1, 1]","Fermionic Operator\nnumber spin orbitals=4, nu...",UCCSD,PauliTwoDesign,SPSA,numpy_solver,gses


# 2-2. Construct Problem Unittest

In [ ]:
def get_struct_unittest(molecule, lst_mapper_type, lst_reduced, lst_solver_type, lst_method):
    test_dict = {}
    
    for a in lst_mapper_type:
        for b in lst_reduced:
            for c in lst_solver_type:
                for d in lst_method:
                    try:
                        # print('Now running: {}|{}|{}|{}'.format(a, b, c, d))
                        fermionic_hamiltonian, num_particles, num_spin_orbitals, qubit_op, qubit_converter, ground_state, energy = get_construct_problem(molecule, hyperparam,
                                                                                                                                 reduced = b, 
                                                                                                                                 basis = 'sto3g', 
                                                                                                                                 mapper_type = a, 
                                                                                                                                 solver_type = c, 
                                                                                                                                 method = d, 
                                                                                                                                 display_report = False)
                        test_dict['{}|{}|{}|{}'.format(a, b, c, d)] = energy
                    except:
                        test_dict['{}|{}|{}|{}'.format(a, b, c, d)] = 'Error'
                        
    return test_dict

In [ ]:
lst_mapper_type = ['JordanWignerMapper']
lst_reduced = ['']
lst_solver_type = ['numpy_solver_with_filter', 'numpy_solver', 'vqe_ansatz', 'vqe_initial_circuit', 'vqe_qnspsa', 'vqe_runtime', 'vqe_runtime_with_noise']
lst_method = ['gses', 'qpe']

gas_molecules = {
    'h2': {'symbols': ["H", "H"],
           'coords': [(0.0000, 0.0000, 0.0000),
                      (0.0000, 0.0000, 0.7414)],
           'multiplicity': 1,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [1, 1],
           'atom_pair': (1, 2),
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           }
}

hyperparam = {
    'tl_reps': 2,
    'two_qubit_reduction': True,
    'z2symmetry_reduction': 'auto',
    'perturbation_steps': np.linspace(-0.5, 0.5, 250),
    'qpe_num_time_slices': 1,
    'qpe_n_ancilliae': 3,

}

get_struct_unittest(molecule = gas_molecules['h2'], lst_mapper_type = lst_mapper_type, lst_reduced = lst_reduced, lst_solver_type = lst_solver_type, lst_method = lst_method)

# ======================================

# 3-1. Calculate molecule energy by BOPES calculations

In [ ]:
# return fermionic_hamiltonian, num_particles, num_spin_orbitals, qubit_op, qubit_converter, ground_state

def get_molecule_bopes(molecule, hyperparam, reduced = 'ActiveSpace', basis = 'sto3g', 
                       mapper_type = 'JordanWignerMapper', solver_type = 'numpy_solver', method = 'gses', 
                       perturbation_steps = np.linspace(-3, 3, 250), display_report = False):
    
    # Obtain molecule coordinates
    # ===========================
    EPSILON = 1e-3
    
    moleculeinfo = MoleculeInfo(symbols = molecule['symbols'], coords = molecule['coords'], charge = molecule['charge'], multiplicity = molecule['multiplicity'], atom_pair = moleculeinfo['atom_pair'])
    x0, y0, z0 = moleculeinfo['coords'][moleculeinfo['atom_pair'][0] - 1]
    x1, y1, z1 = moleculeinfo['coords'][moleculeinfo['atom_pair'][1] - 1]
    
    # Get a straight line connecting the 2 interacting atom pair, y = mx + p
    m = 0
    p = y0
    if abs(x1 - x0) > EPSILON:
        m = (y1 - y0)/(x1 - x0)
        p = y0 - m*x0
    
    # This perturbation assumes lying on the same plane
    size = len(perturbation_steps)
    energy_lst = np.empty(size)
    
    for k in range(size):
        print("Step: ", k)
        
        if (abs(x0) < EPSILON and abs(y0) < EPSILON):
            z0_new = z0 + perturbation_steps[k]
            
            coords_new = []
            for l in range(len(moleculeinfo['coords'])):
                if l == atom_pair[0]:
                    coords_new.append((0.0, 0.0, z0_new))
                else:
                    coords_new.append(moleculeinfo['coords'][l])
        
        elif (abs(z0) < EPSILON and abs(z1) < EPSILON):
            x0_new = x0 + perturbation_steps[k]
            y0_new = m*x0_new + p
            
            coords_new = []
            for l in range(len(moleculeinfo['coords'])):
                if l == atom_pair[0]:
                    coords_new.append((x0_new, y0_new, 0.0))
                else:
                    coords_new.append(moleculeinfo['coords'][l])
                    
        else:
            print("bopes - Error: unsupported molecule geometry, atom pairs must be in the same line or in the same plane")
            return perturbation_steps, 0
    
        moleculeinfo_new = moleculeinfo.copy()
        moleculeinfo_new['coords'] = coords_new
        
        # Get ground state energy from construct problem
        # ===========================
        ground_state, energy, display_df = get_construct_problem(molecule, hyperparam, display_report = True, reduced = 'ActiveSpace', basis = 'sto3g', 
                      mapper_type = 'JordanWignerMapper', solver_type = 'numpy_solver', method = 'gses', seed = _SEED)
        energy_lst[k] = energy
        
        if display_report:
            if len(perturbation_steps) > 1:
                plt.plot(perturbation_steps, energy_lst, label="VQE Energy")
                plt.xlabel('Atomic distance Deviation(Angstrom)')
                plt.ylabel('Energy (hartree)')
                plt.legend()
                plt.show()
            else:
                # print("Total Energy is: ", energy_surface_result.energies[0], "hartree")
                print("(No need to plot, only one configuration calculated.)")
        
        return perturbation_steps, energy_lst


# 3-2. BOPES Unittest

In [ ]:
def get_bopes_unittest():
    pass

In [ ]:
gas_molecules = {
    'h2': {'symbols': ["H", "H"],
           'coords': [(0.0000, 0.0000, 0.0000),
                      (0.0000, 0.0000, 0.7414)],
           'multiplicity': 1,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [1, 1],
           'atom_pair': (1, 2),
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           }
}

hyperparam = {
    'tl_reps': 2,
    'two_qubit_reduction': True,
    'z2symmetry_reduction': 'auto',
    'perturbation_steps': np.linspace(-0.5, 0.5, 250),
    'qpe_num_time_slices': 3,
    'qpe_n_ancilliae': 1,

}

In [ ]:

molecule_h2o = {
    'symbols': ['O', 'H', 'H'],
    'coords': [(0.0, 0.0, 0.0), (0.758602, 0.0, 0.504284), (0.758602, 0.0, -0.504284)],
    'multiplicity': 1,
    'charge': 0,
    'units': DistanceUnit.ANGSTROM,
    'masses': [15.999, 1.00784, 1.00784],
    'atom_pair': (1, 0)
}

fermionic_hamiltonian, num_particles, num_spin_orbitals, qubit_op, qubit_converter, ground_state = get_construct_problem(molecule = molecule_h2o, 
                                                                                                                         reduced = 'ActiveSpace', 
                                                                                                                         basis = 'sto3g', 
                                                                                                                         mapper_type = 'ParityMapper', 
                                                                                                                         solver_type = 'numpy_solver', 
                                                                                                                         method = 'gses', 
                                                                                                                         display_report = False)

hyperparam = {
    'fc_remove_orbitals': [4, 5],
    'as_num_electrons': 0,
    'as_num_spatial_orbitals': 0,
    'as_active_orbitals': None,
    'tl_reps': 2,
    'two_qubit_reduction': True,
    'z2symmetry_reduction': 'auto',
}

# Execution

In [ ]:
gas_molecules = {
    'h2': {'symbols': ["H", "H"],
           'coords': [(0.0000, 0.0000, 0.0000),
                      (0.0000, 0.0000, 0.7414)],
           'multiplicity': 1,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [1, 1],
           'atom_pair': (1, 2),
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           },
    'n2': {'symbols': ["N", "N"],
           'coords': [(0.0000, 0.0000, 0.5488), 
                      (0.0000, 0.0000, -0.5488)],
           'multiplicity': 4,
           'charge': 0,
           'units': DistanceUnit.ANGSTROM,
           'masses': [7, 7],
           'atom_pair': (1, 2),
           'fc_transformer': {
               'fc_freeze_core': True, 
               'fc_remove_orbitals': None,
               },
           'as_transformer': {
               'as_num_electrons': 0,
               'as_num_spatial_orbitals': 2,
               'as_active_orbitals': [1, 1],
               }
           
           
           },
    'co2': {'symbols': ["C", "O", "O"],
            'coords': [(0.0000, 0.0000, 0.0000),
                       (0.0000, 0.0000, 1.1621),
                       (0.0000, 0.0000, -1.1621)],
            'multiplicity': 2,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'h2o': {'symbols': ["O", "H", "H"],
            'coords': [(0.0000, 0.0000, 0.1173),
                       (0.0000, 0.7572, -0.4692),
                       (0.0000, -0.7572, -0.4692)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'so2': {'symbols': ["S", "O", "O"],
            'coords': [(0.0000, 0.0000, 0.0000),
                       (0.0000, 1.2371, 0.7215),
                       (0.0000, -1.2371, 0.7215)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'no2': {'symbols': ["N", "O", "O"],
            'coords': [(0.0000, 0.0000, 0.0000),
                       (0.0000, 1.0989, 0.4653),
                       (0.0000, -1.0989, 0.4653)],
            'multiplicity': 2,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'n2o': {'symbols': ["N", "N", "O"],
            'coords': [(0.0000, 0.0000, -1.1998),
                       (0.0000, 0.7572, -0.0716),
                       (0.0000, -0.7572, 1.126)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 8, 8],
            'atom_pair': (1, 2)
           },
    'ch4': {'symbols': ["C", "H", "H", "H", "H"],
            'coords': [(0.0000, 0.0000, 0.0000), 
                       (0.6276, 0.6276, 0.6276),
                       (0.6276, -0.6276, -0.6276),
                       (-0.6276, 0.6276, -0.6276),
                       (0.6276, -0.6276, 0.6276)],
            'multiplicity': 1,
            'charge': 0,
            'units': DistanceUnit.ANGSTROM,
            'masses': [12, 1, 1, 1, 1],
            'atom_pair': (1, 4)
           },
    

}

# Not updated
# Spin multiplicity check SO2, N2O, CH4
